In [77]:
# default_exp motor

In [1]:
# just removing the insecure warning for now
# TODO: Secure requests and remove this code
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
from private import server_vars

In [3]:
base_url=server_vars['base_url']
login_email = server_vars['login_email']
login_pwd = server_vars['login_pwd']

In [24]:
import requests

#from yx_motor import logger, config, version

default_headers = {"Content-Type": "application/json",
                    "Accept": "*/*",
                    "Accept-Language": "en-US,en;q=0.5",
                    "Accept-Encoding": "gzip,deflate"
                  }


class API(object):
    def __init__(self, api_url, headers=None):
        self.api_url = api_url
        headers = headers or default_headers
        self.headers = headers.copy()

    def get_path(self, url):
        api_url = self.api_url if not self.api_url.endswith("/") else self.api_url[:-1]
        template = "{}{}" if url.startswith("/") else "{}/{}"
        return template.format(api_url, url)

    def post(self, url, json=None, params=None, files=None, data=None, cookies=None, verify=False):
        path = self.get_path(url)
#         logger.debug("POST request sent to: {} \n\theaders: {}\n\tjson: {}\n\tparams: {}\n\tfiles: {}\n\tdata: {}"
#                      .format(path, self.headers, json, params, files, data))
        response = requests.post(path, 
                                 json=json, 
                                 params=params, 
                                 headers=self.headers, 
                                 files=files, 
                                 data=data, 
                                 cookies=cookies, 
                                 verify=verify)
#         logger.debug("Response status code: {}".format(response.status_code))
#         logger.debug("Response content: {}".format(response.content))
        return response

    def put(self, url, json=None, params=None):
        path = self.get_path(url)
        logger.debug("PUT request sent to: {} \n\theaders: {}\n\tjson: {}\n\tparams: {}"
                     .format(path, self.headers, json, params))
        response = requests.put(path, json=json, params=params, headers=self.headers)
#         logger.debug("Response status code: {}".format(response.status_code))
#         logger.debug("Response content: {}".format(response.content))
        return response

    def get(self, url, json=None, params=None):
        path = self.get_path(url)
#         logger.debug("GET request sent to: {} \n\theaders: {}\n\tjson: {}\n\tparams: {}"
#                      .format(path, self.headers, json, params))
        response = requests.get(path, params=params, headers=self.headers, json=json)
#         logger.debug("Response status code: {}".format(response.status_code))
#         logger.debug("Response content: {}".format(response.content))
        return response

    def delete(self, url, json=None, params=None):
        path = self.get_path(url)
        response = requests.delete(path, params=params, headers=self.headers, json=json)
#         logger.debug("DELETE request sent to: {} \n\theaders: {}\n\tjson: {}\n\tparams: {}"
#                      .format(response.url, self.headers, json, params))
#         logger.debug("Response status code: {}".format(response.status_code))
#         logger.debug("Response content: {}".format(response.content))
        return response

In [44]:
motor.api.get_path('authenticate')

'https://test.aep-mono.devops.alteryx.com/api/v1/authenticate'

In [46]:
# export
import requests

class Motor:
    "Wrapper for Alteryx Server API."
    
    def __init__(self, 
                 base_url: str, 
                 login_email: str, 
                 login_pwd: str):
        
        self.base_url = base_url
        self.api_url = f"{base_url}api/v1/"
        
        self.auth_endpoint = "authenticate"
        
        self.login_email = login_email
        self.login_pwd = login_pwd
        
        self.headers = {
            "Content-Type": "application/json",
            "Accept": "*/*",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip,deflate"
        }
        
        self.jar = requests.cookies.RequestsCookieJar()
        
        self.api = API(api_url=self.api_url, 
                       headers=self.headers)
        
        self.is_authenticated = False
        self.authenticate()
        
        
    def authenticate(self) -> requests.Response:
        payload = {
            "email": self.login_email, 
            "password": self.login_pwd
        }
        response = self.api.post(url=self.auth_endpoint, 
                                 json=payload, 
                                 cookies=self.jar, 
                                 verify=False)
        if response.status_code == 200:
            self.jar.update(response.cookies)
            self.is_authenticated = True
        return response
        
    def get_jobs(self,jobid=None,**kwargs):
        #TODO: API class handles url generation for us, fix this
        response = requests.get(f"{self.base_url}api/v1/jobs",
                     cookies=self.jar,
                     verify=False)
        return response
    
    def get_users(self):
        response = requests.get(f"{self.base_url}api/v1/users",
                               cookies=self.jar,
                               verify=False)
        return response
    
    def get_job(self,jobid):
        response = requests.get(f"{self.base_url}api/v1/jobs/{jobid}",
                     cookies=self.jar,
                     verify=False)
        return response
    
    def get_logs(self,jobid):
        response = requests.get(f"{self.base_url}api/v1/jobs/{jobid}/logs",
                     cookies=self.jar,
                     verify=False)
        return response
    
    def get_job_info(self,jobid,req_type=""):
        response = requests.get(f"{self.base_url}api/v1/jobs/{jobid}/{req_type}",
                     cookies=self.jar,
                     verify=False)
        return response

In [47]:
motor = Motor(base_url,
              login_email, 
              login_pwd)

In [49]:
motor.is_authenticated

True

In [13]:
response1 = motor.get_jobs()

In [14]:
response1.request.headers

{'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Cookie': 'ayxSession=s%3A74147e38-1c76-4767-8d47-f4f259f345c5.ck7%2B7H3r0TgI7BC6j8wRLtdtPI2v6MqHGOIynnzY0AQ'}

In [8]:
x = response1.json()

for i in x['jobs']:
    print(motor.get_job_info(i['jobId'],"logs").json())

{'jobId': 'a829c8c6-6143-4a95-aafa-332d842bd210', 'name': 'SeededOracleWorkflow.yxmd', 'entries': [{'toolId': '-', 'logType': 'worker', 'details': 'Retrieved file simple_questions.yxwz'}, {'toolId': '-', 'logType': 'worker', 'details': 'Engine run started'}, {'toolId': '15', 'logType': '1', 'details': '1 record was output'}, {'toolId': '23', 'logType': '9', 'details': 'simple_question_outputs.csv; 1 records were written to "simple_question_outputs.csv"'}, {'toolId': '-', 'logType': 'worker', 'details': 'Engine run finished in 3.75 seconds'}, {'toolId': '-', 'logType': 'worker', 'details': 'Uploaded output file simple_question_outputs.csv'}], 'completionDate': '2020-04-06T10:20:01.995Z'}
{'jobId': '3077026f-16a7-4e90-8ad5-dc2d9c035cb7', 'name': 'NewWorkflow1.yxmd', 'entries': [{'toolId': '-', 'logType': 'worker', 'details': 'Retrieved file NewWorkflow1.yxmd'}, {'toolId': '-', 'logType': 'worker', 'details': 'Engine run started'}, {'toolId': '-', 'logType': 'worker', 'details': 'Engine r

In [9]:
response2 = motor.get_users()
y = response2.json()
for i in y[:10]:
    print(i['id'])

026b73ba-141f-49a1-8697-c612bc98c6bc
04c256a8-8e93-4965-94c4-6f18530c1f6c
0a9da252-f518-464c-8e61-15aa0d67f5c7
0ad44f3b-f4b9-49ea-89f8-3d0c2c94db8e
0bc15744-dbad-4a6e-9a82-97458f8ce6d2
1063e92c-3ace-4320-9abe-3e7fbafb4863
121c0fec-2dc7-4aa6-a3ee-9c2ea20e9acd
1c7a3e98-91e2-4f29-a870-b569d0df73dd
1cfbfe39-da94-4b98-b397-9201adda40e0
240dc394-4ab0-4826-9db9-929e1013b7f5
